# Data collection and cleaning

**Goal:** gather the different data sources to get a dataset including around 10 features and at least 1000 entries

**Data Preparation tasks:**
- [x] Agreggation of data sources and cleaning of columns names
- [x] Merging data sources
- [x] Checking missing values and interpolation
- [x] Creating calculated columns
- [ ] Add web-scraping of doctolib (?)

In [ ]:
import pandas as pd
import numpy as np

pd.set_option('max_columns',35)

In [ ]:
# Reading the dataset
apl = pd.read_excel('raw_data/apl-drees.xlsx',sheet_name='APL_2018',header=7, index_col=None)
print("Shape:",apl.shape)
apl.head()

In [ ]:
# Dropping first row and last 2 columns that we won't use
apl.drop(0,axis=0, inplace=True)
apl.drop(apl.iloc[:,-2:],axis=1, inplace=True)

In [ ]:
# Cleaning of columns names to make it easier to work
apl = apl.rename(columns={'Code commune INSEE':'CODGEO','APL aux médecins généralistes':'APL'})

# Converting dtype of CODGEO to ensure future merge
apl.CODGEO = apl.CODGEO.astype(str)
apl = apl.convert_dtypes()
apl.info()

### Secondary data source: communes comparateur

Features to get from this data source: 
- Population
- Density area (hab/Km2 - Population/Superficie)
- Population growth
- Mediane Niveau de vie
- Unemployement Rate
- Part of secondary residences (%)
- Part of vacant residences (%)
- Part of city amenities (commerces, services, transports - %)
- Part of city amenities (administration, social, health, education - %)

In [ ]:
# Reading dataset
com = pd.read_excel('raw_data/base_cc_comparateur.xls',sheet_name='COM',header=5)
print("Shape:",com.shape)
com.head()

Information on selected columns:
- P16_POP	    Population en 2016
- P11_POP	    Population en 2011
- SUPERF	    Superficie (en km2)
- NAIS1116	    Nombre de naissances entre le 01/01/2011 et le 01/01/2016
- P16_LOG	    Nombre de logements en 2016
- P16_RSECOCC	Rés secondaires et logts occasionnels en 2016
- P16_LOGVAC	Logements vacants en 2016
- MED16	        Médiane du niveau vie en 2016
- P16_POP1564	Nombre de personnes de 15 à 64 ans en 2016
- P16_CHOM1564	Nombre de chômeurs de 15 à 64 ans en 2016
- ETTOT15	    Total des établissements actifs au 31 décembre 2015
- ETGU15	    Établissements actifs du commerce, transports et services divers au 31/12/2015
- ETOQ15	    Ets actifs de l'administration publique au 31/12/2015

In [ ]:
# Selecting the interesting columns
sub_com = com[['CODGEO','P16_POP','SUPERF','P11_POP','P16_CHOM1564','P16_POP1564','P16_RSECOCC','P16_LOG',
     'P16_LOGVAC','ETGU15','ETOQ15','ETTOT15','MED16','NAIS1116']]

# Converting dtypes to ensure matching between CODGEO columns - dtype of CODGEO should be string
sub_com = sub_com.convert_dtypes()
sub_com.info()

In [ ]:
# Reading dataset with sheetname for borough
arr = pd.read_excel('raw_data/base_cc_comparateur.xls',sheet_name='ARM',header=5)
print("Shape:",arr.shape)
arr.head()

In [ ]:
# Selecting the interesting columns
sub_arr = arr[['CODGEO','P16_POP','SUPERF','P11_POP','P16_CHOM1564','P16_POP1564','P16_RSECOCC','P16_LOG',
     'P16_LOGVAC','ETGU15','ETOQ15','ETTOT15','MED16','NAIS1116']]

sub_arr.CODGEO = sub_arr.CODGEO.astype(str)

# Converting dtypes to ensure matching between CODGEO columns - dtype of CODGEO should be string
sub_arr = sub_arr.convert_dtypes()
sub_arr.info()

In [ ]:
# Concatenate both dataframes before merging
sub_com_arr = pd.concat([sub_com, sub_arr])

In [ ]:
# Merging the dataframes
df_merged = pd.merge(apl,sub_com_arr,'left', on='CODGEO')
print(df_merged.shape)
df_merged.head()

### Secondary data source: evolution structures

Features to get from this datasource: 
- Repartition of population age
- Mobility_rate (% of population located 1 year ago)
- Socio-Professional Category

In [ ]:
# Reading dataset
evol = pd.read_csv('raw_data/base-cc-evol-struct-pop-2016-csv/base-cc-evol-struct-pop-2016.CSV',sep=';')
print("Shape:",evol.shape)
evol.head()

Information on selected columns:
- P16_POP01P	Nombre de personnes de 1 an ou plus localisée 1 an auparavant en 2016
- P16_POP0014	Pop 0-14 ans en 2016	
- P16_POP1529	Pop 15-29 ans en 2016		
- P16_POP3044	Pop 30-44 ans en 2016		
- P16_POP4559	Pop 45-59 ans en 2016		
- P16_POP6074	Pop 60-74 ans en 2016	
- P16_POP7589	Pop 75-89 ans en 2016
- P16_POP90P	Pop 90 ans ou plus en 2016
- C16_POP15P	Pop 15 ans ou plus en 2016		
- C16_POP15P_CS1	Pop 15 ans ou plus Agriculteurs exploitants en 2016
- C16_POP15P_CS2	Pop 15 ans ou plus Artisans, Comm., Chefs entr. en 2016
- C16_POP15P_CS3	Pop 15 ans ou plus Cadres, Prof. intel. sup. en 2016
- C16_POP15P_CS4	Pop 15 ans ou plus Prof. intermédiaires  en 2016
- C16_POP15P_CS5	Pop 15 ans ou plus Employés en 2016
- C16_POP15P_CS6	Pop 15 ans ou plus Ouvriers en 2016
- C16_POP15P_CS7	Pop 15 ans ou plus Retraités en 2016
- C16_POP15P_CS8	Pop 15 ans ou plus Autres sans activité professionnelle en 2016

In [ ]:
# Selecting the interesting columns
sub_evol = evol[['CODGEO','P16_POP01P']+list(evol.columns[2:9])+list(evol.columns[51:60])]

# Correcting CODGEO that have only 4 number by adding 0 before
sub_evol.CODGEO = sub_evol.CODGEO.apply(lambda x: '0'+str(x) if len(str(x))==4 else x).astype(str).copy()

# Converting dtypes to ensure matching between CODGEO columns - dtype of CODGEO should be string
sub_evol = sub_evol.convert_dtypes()
sub_evol.info()

In [ ]:
# Merging the dataframes
df_merged_2 = pd.merge(df_merged,sub_evol,'left', on='CODGEO')
print(df_merged_2.shape)
df_merged_2.head()

### Secondary data source: equipements

Features to get from this datasource: 
- Level of medical education = number of health education establishment

(other possible features: Number of leisure establishments, Number of healthcare establishments)

In [ ]:
# Reading the dataset
eqmt = pd.read_csv('raw_data/bpe18_ensemble_csv/bpe18_ensemble.csv',sep=';')
print("Shape:",eqmt.shape)
eqmt.head()

In [ ]:
# Aggregating number of health education establishements
education_health = eqmt[eqmt.TYPEQU=='C402'].groupby('DEPCOM').NB_EQUIP.agg('sum').reset_index()

# Renaming DEPCOM as CODGEO to match with merge dataframe
education_health = education_health.rename(columns={'DEPCOM':'CODGEO'})

# Cleaning codes to add 0 when code is only 4 number
education_health.CODGEO = education_health.CODGEO.apply(lambda x: '0'+str(x) if len(str(x))==4 else x).astype(str).copy()

# Converting dtypes to ensure matching between CODGEO columns - dtype of CODGEO should be string
education_health = education_health.convert_dtypes()
education_health.info()

In [ ]:
# Merging the dataframes 
df_merged_3 = pd.merge(df_merged_2, education_health,'left', on='CODGEO')
print(df_merged_3.shape)
df_merged_3.head()

In [ ]:
# Filling nan values of Number of health education establishments because it means there isn't any.
df_merged_3.NB_EQUIP = df_merged_3.NB_EQUIP.fillna(0)

In [ ]:
df_merged_3.info()

In [ ]:
# Saving the dataframe as it is if I want to retreive raw data
df_merged_3.to_csv('data/medical_desert_raw_data.csv',index=False)

________________________
## Cleaning missing values

In [ ]:
df_clean = df_merged_3.copy()

In [ ]:
df_clean.isna().sum()

In [ ]:
mis_value = df_clean[df_clean.MED16.isna()]
mis_value

In [ ]:
df_clean[(df_clean.CODGEO.str.contains("^97",regex=True))&(df_clean.P16_POP<1000)]

In [ ]:
# Creating a function to return similar city within the same department and having same population
# Coefficient attribute allow to get an higher range of similar city, especially for small city
import re

def get_similar(codgeo, pop, coef):
    
    global df_clean
    
    return df_clean[(df_clean.CODGEO.str.contains(f"^{codgeo[:2]}",regex=True))&(df_clean.P16_POP<(pop+coef))]
                           
# Testing the function 
mis_value.apply(lambda x: get_similar(x['CODGEO'],x['P16_POP'],10).MED16.median(),axis=1)
                           

In [ ]:
# Filling missing values with median of similar city - WARNING: Takes time to run
mis_value = df_clean[df_clean.MED16.isna()]

df_clean.MED16 = df_clean.MED16.fillna(mis_value.apply(lambda x: get_similar(x['CODGEO'],x['P16_POP'],200).MED16.median(),axis=1))



In [ ]:
# Doing it again for the 3 last rows by increasing the coefficient of similarity 
# (not enough values with the previous coef)
mis_value = df_clean[df_clean.MED16.isna()]

df_clean.MED16 = df_clean.MED16.fillna(mis_value.apply(lambda x: get_similar(x['CODGEO'],x['P16_POP'],1000).MED16.median(),axis=1))

In [ ]:
# Filling the last 3 missing columns by getting the mode of similar city

missing_col = df_clean.iloc[29876][df_clean.iloc[29876].isna()].index

for col in missing_col:
        df_clean.loc[29876,col] = get_similar(df_clean.iloc[29876]['CODGEO'],
                                              df_clean.iloc[29876]['P16_POP'],200)[col].mode()[0]
                              

In [ ]:
df_clean.isna().sum()

______________________________________
## Calculate new columns

The objective is to create calculated columns based on the data retrieved from the different sources. It will allow to add relative type of data (percentage) to make them more relevant to compare. 

In [ ]:
new_df = df_clean.copy()

In [ ]:
# Calculated metrics
new_df['density_area'] = new_df.P16_POP / new_df.SUPERF
new_df['annual_pop_growth'] = ((new_df.P16_POP/new_df.P11_POP)**(1/(2016-2011))-1)*100
new_df['unemployment_rate'] = (new_df.P16_CHOM1564/new_df.P16_POP1564)*100
new_df['secondary_residence_rate'] = (new_df.P16_RSECOCC/new_df.P16_LOG)*100
new_df['vacant_residence_rate'] = (new_df.P16_LOGVAC/new_df.P16_LOG)*100
new_df['active_local_business_rate'] = (new_df.ETGU15/new_df.ETTOT15)*100
new_df['city_social_amenities_rate'] = (new_df.ETOQ15/new_df.ETTOT15)*100
new_df['0_14_pop_rate'] = (new_df.P16_POP0014/new_df.P16_POP)*100
new_df['15_59_pop_rate'] = ((new_df.P16_POP1529+new_df.P16_POP3044+new_df.P16_POP4559)/new_df.P16_POP)*100
new_df['60+_pop_rate'] = ((new_df.P16_POP6074+new_df.P16_POP7589+new_df.P16_POP90P)/new_df.P16_POP)*100
new_df['mobility_rate'] = ((new_df.P16_POP-new_df.P16_POP01P)/new_df.P16_POP)*100
new_df['average_birth_rate'] = (new_df.NAIS1116/(2016-2011))/(new_df[['P16_POP','P11_POP']].mean(axis=1))*100

for i in range(1,9): 
    new_df[f'CSP{i}_rate'] = ((new_df[f'C16_POP15P_CS{i}']/new_df.C16_POP15P)*100)


new_df = new_df.rename(columns={'MED16':'median_living_standard','NB_EQUIP':'healthcare_education_establishments'})

In [ ]:
new_df = new_df.fillna(0)
#new_df.isna().sum()

In [ ]:
final_df = new_df[list(new_df.columns[:3])+['median_living_standard']+list(new_df.columns[-21:])]
print(final_df.shape)
final_df.head()

In [ ]:
# Saving the clean dataframe
final_df.to_csv('data/medical_desert_clean.csv',index=False)